## Imports

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time

User inputs a search term

In [2]:
driver = webdriver.Chrome()

keywords = 'bar bar black sheep nus'
keywords.replace(" ", "+")

url = 'https://www.google.com/maps/search/' + keywords + '+singapore'
driver.get(url)

In [3]:
# driver.find_element(By.XPATH,'//*[@id="pane"]/div/div[1]/div/div/div[' + str(a) + ']/div/button')
# time.sleep(5)

In [4]:
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.support.ui import WebDriverWait

def check_exists_by_xpath(xpath):
    try:
        wait = WebDriverWait(driver, 5)
        driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        return False
    return True

# xpath = '//*[@id="pane"]/div/div[1]/div/div/div[38]/div/button'
# driver.find_element(By.XPATH, xpath).click()

for i in range(37, 43):
    xpath = '//*[@id="pane"]/div/div[1]/div/div/div['+str(i)+']/div/button'
    # xpath = '//*[@id="pane"]/div/div[1]/div/div/div[38]/div/button'
    
    if check_exists_by_xpath(xpath) == True:
        # driver.find_element_by_xpath(xpath).click()
        driver.find_element(By.XPATH, xpath).click()
        
        break
    else:
        continue
time.sleep(3)

In [5]:
#Find scroll layout
scrollable_div = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')

#Scroll as many times as necessary to load all reviews
for i in range(0,21):
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        time.sleep(1)

<ipython-input-5-31db701e48bc>:2: DeprecationWarning:

find_element_by_* commands are deprecated. Please use find_element() instead



In [6]:
from parsel import Selector

page_content = driver.page_source
response = Selector(page_content)

results = []

for el in response.xpath('//div/div[@data-review-id]/div[contains(@class, "content")]'):
    results.append({
        'title': el.xpath('.//div[contains(@class, "title")]/span/text()').extract_first(''),
        'rating': el.xpath('.//span[contains(@aria-label, "stars")]/@aria-label').extract_first('').replace('stars' ,'').strip(),
        'body': el.xpath('.//span[contains(@class, "text")]/text()').extract_first(''),
    })

#print(results)

In [7]:
import pandas as pd

df = pd.DataFrame(results)
df.dropna()
df['rating'] = pd.to_numeric(df['rating'])
df.head()

,title,rating,body
0,BOB LEE,5.0,Good vibe! Nice place to have a tasty meal! Wi...
1,Maybelline Ooi Sim Leng,5.0,Chef Ae & Elsie made us feel so at home being ...
2,bill jova,5.0,"Nice place for after work, relax Soul and stom..."
3,ally eric,5.0,Shared a bottle of Four Pillars with some frie...
4,Alan K (AK),5.0,Great spot for a quick beer or two. Didn't hav...


In [8]:
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud

stopwords = set(stopwords.words('english'))
stopwords.update(["campus", "NUS", "MRT", "looks", "food"])
textt = " ".join(review for review in df['body'])
wordcloud = WordCloud(stopwords=stopwords).generate(textt)
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.savefig('wordcloud11.png')
# plt.show()

In [9]:
# assign reviews with score > 3 as positive sentiment
# score < 3 negative sentiment

df['sentiment'] = df['rating'].apply(lambda score : +1 if score > 3.5 else -1)

In [10]:
positive = df[df['sentiment'] == 1]
negative = df[df['sentiment'] == -1]

In [11]:
#stopwords = set(stopwords.words('english'))
stopwords.update(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't", "...", "-"]) 

pos = " ".join(review for review in positive['body'])
wordcloud2 = WordCloud(stopwords=stopwords).generate(pos)
# plt.imshow(wordcloud2, interpolation='bilinear')
# plt.axis("off")
# plt.show()

In [12]:
pos = pos.split()
for word in list(pos):  # iterating on a copy since removing will mess things up
    if word in stopwords:
        pos.remove(word)

from collections import Counter
counts = Counter(pos)

In [13]:
top_words = counts.most_common(5)
top_words

[('place', 11), ('Great', 10), ('Nice', 7), ('great', 7), ('good', 7)]

In [14]:
neg = " ".join(review for review in negative['body'])
wordcloud3 = WordCloud(stopwords=stopwords).generate(neg)
# plt.imshow(wordcloud3, interpolation='bilinear')
# plt.axis("off")
# plt.savefig('wordcloud33.png')
# plt.show()

In [15]:
df['sentiment'] = df['sentiment'].replace({-1 : 'negative'})
df['sentiment'] = df['sentiment'].replace({1 : 'positive'})
# fig = px.histogram(df, x="sentiment")
# fig.update_traces(marker_color="indianred",marker_line_color='rgb(8,48,107)',
#                   marker_line_width=1.5)
# fig.update_layout(title_text='Product Sentiment')
# fig.show()

In [16]:
score = df['sentiment'].value_counts()['positive'] / (df['sentiment'].value_counts()['negative'] + df['sentiment'].value_counts()['positive']) * 100
score

73.77049180327869